# 長文コンテンツ抽出

GPT-3は、コンテキストウィンドウに収まりきらない大きなドキュメントから主要な数字、日付、またはその他の重要なコンテンツを抽出するのに役立ちます。これを解決するための一つのアプローチは、ドキュメントをチャンクに分割し、各チャンクを個別に処理してから、回答のリストに結合することです。

このノートブックでは、次のアプローチを実行します：
- 長いPDFを読み込んでテキストを取得する
- 重要な情報を抽出するために使用されるプロンプトを作成する
- ドキュメントをチャンクに分割し、各チャンクを処理して回答を抽出する
- 最後にそれらを結合する
- このシンプルなアプローチは、より難しい質問に拡張されます

## アプローチ

- **セットアップ**: Formula 1 Financial RegulationドキュメントのPower Unitsに関するPDFを取得し、エンティティ抽出のためにそのテキストを抽出します。これを使用して、コンテンツに埋もれた回答を抽出しようとします。
- **シンプルなエンティティ抽出**: ドキュメントのチャンクから重要な情報を抽出するために：
    - 質問と期待されるフォーマットの例を含むテンプレートプロンプトを作成します。
    - テキストのチャンクを入力として受け取り、プロンプトと組み合わせて応答を取得する関数を作成します。
    - スクリプトを実行してテキストをチャンクに分割し、回答を抽出し、解析のためにそれらを出力します。
- **複雑なエンティティ抽出**: より難解な質問をし、より困難な推論が必要な質問をします。

## Setup

In [ ]:
!pip install textract
!pip install tiktoken

In [5]:
import textract
import os
import openai
import tiktoken

# Extract the raw text from each PDF using textract
text = textract.process('data/fia_f1_power_unit_financial_regulations_issue_1_-_2022-08-16.pdf', method='pdfminer').decode('utf-8')
clean_text = text.replace("  ", " ").replace("\n", "; ").replace(';',' ')

## Simple Entity Extraction

In [28]:
# Example prompt - 
document = '<document>'
template_prompt=f'''Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output \"Not specified\".
When you extract a key piece of information, include the closest page number.
Use the following format:\n0. Who is the author\n1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR\n2. What is the value of External Manufacturing Costs in USD\n3. What is the Capital Expenditure Limit in USD\n\nDocument: \"\"\"{document}\"\"\"\n\n0. Who is the author: Tom Anderson (Page 1)\n1.'''
print(template_prompt)
# document = '<document>'
# template_prompt = f'''この規制文書から重要な情報を抽出してください。
# 特定の情報が存在しない場合、"指定されていない"と出力してください。
# 重要な情報を抽出する際には、最も近いページ番号を含めてください。
# 以下のフォーマットを使用してください：
# 0. 著者は誰ですか？
# 1. "Power Unit Cost Cap"の金額はUSD、GBP、EURでいくらですか？
# 2. 外部製造コストの金額はUSDでいくらですか？
# 3. 資本支出制限はUSDでいくらですか？

# 文書: \"\"\"{document}\"\"\"

# 0. 著者は誰ですか？：トム・アンダーソン（ページ1）
# 1. "Power Unit Cost Cap"の金額はUSD、GBP、EURでいくらですか？：（ページ番号）
# 2. 外部製造コストの金額はUSDでいくらですか？：（ページ番号）
# 3. 資本支出制限はUSDでいくらですか？：（ページ番号）
# '''
# print(template_prompt)


Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output "Not specified".
When you extract a key piece of information, include the closest page number.
Use the following format:
0. Who is the author
1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR
2. What is the value of External Manufacturing Costs in USD
3. What is the Capital Expenditure Limit in USD

Document: """<document>"""

0. Who is the author: Tom Anderson (Page 1)
1.


In [29]:
# テキストをサイズnの小さなチャンクに分割し、可能であれば文の終わりで終了することをお勧めします。
def create_chunks(text, n, tokenizer):
    tokens = tokenizer.encode(text)
    """テキストから連続するnサイズのチャンクを生成します。"""
    i = 0
    while i < len(tokens):
        # 範囲0.5 * nから1.5 * nトークン内で最寄りの文末を見つけます。
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # トークンをデコードし、句点または改行をチェックします。
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith("。") or chunk.endswith("\n"):
                break
            j -= 1
        # 文末が見つからない場合、チャンクサイズとしてnトークンを使用します。
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j

def extract_chunk(document,template_prompt):
    
    prompt=template_prompt.replace('<document>',document)

    response = openai.Completion.create(
    model='text-davinci-003', 
    prompt=prompt,
    temperature=0,
    max_tokens=1500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )
    return "1." + response['choices'][0]['text']


In [ ]:
# Initialise tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

results = []
    
chunks = create_chunks(clean_text,1000,tokenizer)
text_chunks = [tokenizer.decode(chunk) for chunk in chunks]

for chunk in text_chunks:
    results.append(extract_chunk(chunk,template_prompt))
    #print(chunk)
    print(results[-1])


In [31]:
groups = [r.split('\n') for r in results]

# zip the groups together
zipped = list(zip(*groups))
zipped = [x for y in zipped for x in y if "Not specified" not in x and "__" not in x]
zipped

['1. What is the amount of the "Power Unit Cost Cap" in USD, GBP and EUR: USD 95,000,000 (Page 2); GBP 76,459,000 (Page 2); EUR 90,210,000 (Page 2)',
 '2. What is the value of External Manufacturing Costs in USD: US Dollars 20,000,000 in respect of each of the Full Year Reporting Periods ending on 31 December 2023, 31 December 2024 and 31 December 2025, adjusted for Indexation (Page 10)',
 '3. What is the Capital Expenditure Limit in USD: US Dollars 30,000,000 (Page 32)']

## Complex Entity Extraction

In [32]:
# Example prompt - 
template_prompt=f'''Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output \"Not specified\".
When you extract a key piece of information, include the closest page number.
Use the following format:\n0. Who is the author\n1. How is a Minor Overspend Breach calculated\n2. How is a Major Overspend Breach calculated\n3. Which years do these financial regulations apply to\n\nDocument: \"\"\"{document}\"\"\"\n\n0. Who is the author: Tom Anderson (Page 1)\n1.'''
print(template_prompt)

# template_prompt = f'''この規制文書から重要な情報を抽出してください。
# 特定の情報が存在しない場合は、「指定されていない」と出力してください。
# 重要な情報を抽出する際に、最も近いページ番号を含めてください。
# 以下のフォーマットを使用してください：
# 0. 著者は誰ですか？
# 1. 小規模な超過違反はどのように計算されますか？
# 2. 大規模な超過違反はどのように計算されますか？
# 3. これらの財務規制はどの年度に適用されますか？

# 文書: \"\"\"{document}\"\"\"

# 0. 著者: トム・アンダーソン（ページ1）
# 1. ...
# '''
# print(template_prompt)


Extract key pieces of information from this regulation document.
If a particular piece of information is not present, output "Not specified".
When you extract a key piece of information, include the closest page number.
Use the following format:
0. Who is the author
1. How is a Minor Overspend Breach calculated
2. How is a Major Overspend Breach calculated
3. Which years do these financial regulations apply to

Document: """<document>"""

0. Who is the author: Tom Anderson (Page 1)
1.


In [34]:
results = []

for chunk in text_chunks:
    results.append(extract_chunk(chunk,template_prompt))
    
groups = [r.split('\n') for r in results]

# zip the groups together
zipped = list(zip(*groups))
zipped = [x for y in zipped for x in y if "Not specified" not in x and "__" not in x]
zipped

['1. How is a Minor Overspend Breach calculated: A Minor Overspend Breach arises when a Power Unit Manufacturer submits its Full Year Reporting Documentation and Relevant Costs reported therein exceed the Power Unit Cost Cap by less than 5% (Page 24)',
 '2. How is a Major Overspend Breach calculated: A Material Overspend Breach arises when a Power Unit Manufacturer submits its Full Year Reporting Documentation and Relevant Costs reported therein exceed the Power Unit Cost Cap by 5% or more (Page 25)',
 '3. Which years do these financial regulations apply to: 2026 onwards (Page 1)',
 '3. Which years do these financial regulations apply to: 2023, 2024, 2025, 2026 and subsequent Full Year Reporting Periods (Page 2)',
 '3. Which years do these financial regulations apply to: 2022-2025 (Page 6)',
 '3. Which years do these financial regulations apply to: 2023, 2024, 2025, 2026 and subsequent Full Year Reporting Periods (Page 10)',
 '3. Which years do these financial regulations apply to: 202

## 統合

私たちは最初の2つの回答を安全に抽出することができましたが、3つ目の回答は各ページに表示されていた日付に混乱しました。ただし、正しい回答も含まれています。

さらに調整するために、以下の実験を考えてみることができます：
- より具体的または特定のプロンプト
- 十分なトレーニングデータがある場合、非常に良い出力のセットを見つけるためにモデルを微調整
- データをチャンクに分割する方法 - 私たちは1000トークンでオーバーラップなしでチャンクを作成しましたが、情報をセクションに分割したり、トークンでカットしたりするより賢いチャンキングがより良い結果を得るかもしれません。

しかし、最小限の調整で、長いドキュメントの内容を使用して難易度の異なる6つの質問に答えることができ、エンティティの抽出を必要とする任意の長いドキュメントに適用できる再利用可能なアプローチを持っています。これをどのように活用できるか楽しみにしています！